In [ ]:
import math
from smt.sampling_methods import LHS
import torch              
import torch.nn as nn                                   
from collections import OrderedDict
import numpy as np

In [121]:
# CUDA GPU selection
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [122]:
# Definition of Deep Neural Network Architecture
class DNNModel(torch.nn.Module):
    def __init__(self,input_size,hidden_size,output_size,depth,act=torch.nn.Tanh,init_mode='xavier'):
        super(DNNModel, self).__init__()
        
        layers = [('input', torch.nn.Linear(input_size, hidden_size))]
        layers.append(('input_activation', act()))
        for i in range(depth): 
            layers.append(
                ('hidden_%d' % i, torch.nn.Linear(hidden_size, hidden_size))
            )
            layers.append(('activation_%d' % i, act()))
        layers.append(('output', torch.nn.Linear(hidden_size, output_size)))

        layerDict = OrderedDict(layers)
        self.layers = torch.nn.Sequential(layerDict)

        if init_mode == 'xavier':
          for m in self.modules():                
              if isinstance(m, nn.Linear):
                  size = m.weight.size()
                  fan_out = size[0] # Number of rows
                  fan_in = size[1] # Number of columns
                  variance = math.sqrt(2.0/(fan_in+fan_out))
                  m.weight.data.normal_(0.0, variance)
                  if m.bias is not None:
                    m.bias.data.normal_(0, variance)
        

    def forward(self, x):
        out = self.layers(x)
        return out

In [123]:
# Defintion of Physics infomed neural network
class PINN():
    def __init__(self):
        
        self.model = DNNModel(
            input_size=2,
            hidden_size=50,
            output_size=5,
            depth=8,
            act=torch.nn.Tanh,
            init_mode='xavier'
            ).to(device)
        
        self.dx = 0.01        
        self.dy = 0.005
        self.dx1 = 0.005
        self.dy1 = 0.01

        self.nx = 300
        self.ny = 200
        self.nx1 = 200
        self.ny1 = 100
        self.nx2 = 100
        
        self.rho=1.1644
        self.mu=0.005        
      
        # Bounday collocation points
        xb = torch.arange(0, 3 + self.dx, self.dx)        

        xu1 = torch.arange(0, 1 + self.dx, self.dx)      

        xu2 = torch.arange(2, 3 + self.dx, self.dx)  

        yb = torch.arange(0, 1 + self.dy, self.dy) 

        xb1 = torch.arange(1, 2 + self.dx1, self.dx1)

        yb1 = torch.arange(1, 2 + self.dy1, self.dy1) 

        # Boundary discretization
        self.inlet=torch.stack(torch.meshgrid(torch.zeros(1), yb)).reshape(2, -1).T
        self.inlet1=torch.stack(torch.meshgrid(torch.ones(1)*3, yb)).reshape(2, -1).T
        
        self.outlet=torch.stack(torch.meshgrid(xb1, torch.ones(1)*2)).reshape(2, -1).T
        
        bc1 = torch.stack(torch.meshgrid(xb, torch.zeros(1))).reshape(2, -1).T
        bc2 = torch.stack(torch.meshgrid(xu1, torch.ones(1))).reshape(2, -1).T
        bc3 = torch.stack(torch.meshgrid(xu2, torch.ones(1))).reshape(2, -1).T
        bc4 = torch.stack(torch.meshgrid(torch.ones(1), yb1)).reshape(2, -1).T
        bc5 = torch.stack(torch.meshgrid(torch.ones(1)*2, yb1)).reshape(2, -1).T

        self.bc = torch.cat([bc1, bc2, bc3, bc4, bc5])
        
        # Domain collocation points
        xlimits = np.array([[0.001, 0.0], [2.999, 0.0]])
        sampling = LHS(xlimits=xlimits)        
        x = sampling(self.nx)
        xc=torch.tensor(x,dtype=torch.float32)
       
        xlimits = np.array([[0.0, 0.001], [0.0, 0.999]])
        sampling = LHS(xlimits=xlimits)        
        y = sampling(self.ny)
        yc=torch.tensor(y,dtype=torch.float32)        

        xlimits = np.array([[0.0, 0.0], [1.001, 1.999]])
        sampling = LHS(xlimits=xlimits)        
        x = sampling(self.nx1)
        x1=torch.tensor(x,dtype=torch.float32)

        xlimits = np.array([[0.0, 0.0], [1.0, 1.999]])
        sampling = LHS(xlimits=xlimits)        
        y = sampling(self.ny1)
        y1=torch.tensor(y,dtype=torch.float32)              

        # Domain discretization
        self.channel = torch.stack(torch.meshgrid(xc[:,1], yc[:,1])).reshape(2, -1).T
        self.channel1 = torch.stack(torch.meshgrid(x1[:,1], y1[:,1])).reshape(2, -1).T
        
        self.tm = torch.cat([self.channel, self.channel1])             
        
        u_inlet = torch.tensor(6*(1-yb)*yb)
        v_inlet = torch.zeros(len(self.inlet))

        u_inlet1 = torch.tensor(-6*(1-yb)*yb)
        v_inlet1 = torch.zeros(len(self.inlet1))

        p_outlet = torch.zeros(len(self.outlet))

        u_bc = torch.zeros(len(self.bc))
        v_bc = torch.zeros(len(self.bc))                

        self.uv_inlet = torch.stack((u_inlet, v_inlet)).reshape(2, -1).T
        self.uv_inlet1 = torch.stack((u_inlet1, v_inlet1)).reshape(2, -1).T
        self.p_outlet = (p_outlet).reshape(1, -1).T

        self.uv_bc = torch.stack((u_bc, v_bc)).reshape(2, -1).T     

        self.pde_sol=torch.zeros(len(self.tm))        
        
        self.tm = self.tm.to(device)
        self.bc = self.bc.to(device)        
        self.inlet = self.inlet.to(device)
        self.inlet1 = self.inlet1.to(device)
        self.outlet = self.outlet.to(device)
        self.uv_inlet = self.uv_inlet.to(device)
        self.uv_inlet1 = self.uv_inlet1.to(device)
        self.p_outlet = self.p_outlet.to(device)
        self.pde_sol = self.pde_sol.to(device)
        self.uv_bc = self.uv_bc.to(device)        
        self.tm.requires_grad = True
        self.bc.requires_grad = True        
        self.inlet.requires_grad = True
        self.inlet1.requires_grad = True
        self.outlet.requires_grad = True
        
        self.criterion = torch.nn.MSELoss()
        self.iter = 1
        
        self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr=0.001, 
            max_iter=100000, 
            max_eval=100000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps
            )
        
        self.adam = torch.optim.Adam(self.model.parameters(), lr=0.0015)
        
    #Definition of Bounday and Navier-stokes residual loss   
    def loss_func(self):
        self.optimizer.zero_grad()
        
        y_inlet = self.model(self.inlet)
        dpsi_dX = torch.autograd.grad(inputs=self.inlet, outputs=y_inlet[:,0], grad_outputs=torch.ones_like(y_inlet[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]    
        
        loss_inlet = self.criterion(dpsi_dy, self.uv_inlet[:,0])+self.criterion(dpsi_dx, self.uv_inlet[:,1])

        y_inlet1 = self.model(self.inlet1)
        
        dpsi_dX = torch.autograd.grad(inputs=self.inlet1, outputs=y_inlet1[:,0], grad_outputs=torch.ones_like(y_inlet1[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]      
        
        loss_inlet1 = self.criterion(dpsi_dy, self.uv_inlet1[:,0])+self.criterion(dpsi_dx, self.uv_inlet1[:,1])   
            
        y_bc = self.model(self.bc)
        
        dpsi_dX = torch.autograd.grad(inputs=self.bc, outputs=y_bc[:,0], grad_outputs=torch.ones_like(y_bc[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]

        loss_bc = self.criterion(dpsi_dy, self.uv_bc[:,0])+self.criterion(dpsi_dx, self.uv_bc[:,1])
        y_outlet = self.model(self.outlet)            

        loss_outlet = self.criterion(y_outlet[:,1], self.p_outlet)
        
        loss_b=loss_inlet+loss_inlet1+loss_bc+loss_outlet

        y_id = self.model(self.tm)

        dpsi_dX = torch.autograd.grad(inputs=self.tm, outputs=y_id[:,0], grad_outputs=torch.ones_like(y_id[:,0]), retain_graph=True, create_graph=True)[0]
        dpsi_dx = -1*dpsi_dX[:, 0] 
        dpsi_dy = dpsi_dX[:, 1]
        dpsi_dXX = torch.autograd.grad(inputs=self.tm, outputs=dpsi_dy, grad_outputs=torch.ones_like(dpsi_dy), retain_graph=True, create_graph=True)[0]
        dpsi_dyx = dpsi_dXX[:, 0] 
        dpsi_dyy = dpsi_dXX[:, 1]
        dpsi_dYY = torch.autograd.grad(inputs=self.tm, outputs=dpsi_dx, grad_outputs=torch.ones_like(dpsi_dx), retain_graph=True, create_graph=True)[0]
        dpsi_dxx = dpsi_dYY[:, 0] 
        dpsi_dxy = dpsi_dYY[:, 1]
        dsxx_dX = torch.autograd.grad(inputs=self.tm, outputs=y_id[:,2], grad_outputs=torch.ones_like(y_id[:,2]), retain_graph=True, create_graph=True)[0]
        dsxx_dx = dsxx_dX[:,0]
        dsyy_dX = torch.autograd.grad(inputs=self.tm, outputs=y_id[:,3], grad_outputs=torch.ones_like(y_id[:,3]), retain_graph=True, create_graph=True)[0]
        dsyy_dy = dsyy_dX[:,1]
        dsxy_dX = torch.autograd.grad(inputs=self.tm, outputs=y_id[:,4], grad_outputs=torch.ones_like(y_id[:,4]), retain_graph=True, create_graph=True)[0]
        dsxy_dx = dsxy_dX[:, 0] 
        dsxy_dy = dsxy_dX[:, 1]       

        loss_u =  self.rho*(dpsi_dy*dpsi_dyx+dpsi_dx*dpsi_dyy)-dsxx_dx-dsxy_dy
        loss_v =  self.rho*(dpsi_dy*dpsi_dxx+dpsi_dx*dpsi_dxy)-dsxy_dx-dsyy_dy
        loss_m =  -1*y_id[:,1]+2*self.mu*(dpsi_dyx+dpsi_dxy)-(y_id[:,2]+y_id[:,3])/2+self.mu*(dpsi_dyy+dpsi_dxx)-y_id[:,4]   

        loss_ns = self.criterion(loss_u+loss_v+loss_m, self.pde_sol)
        b=1.8
        loss = b*loss_b+loss_ns 
        loss.backward()
        if self.iter % 1000 == 0: 
            print(self.iter, loss.item())

        self.iter = self.iter + 1        
        return loss
    
    def train(self):
        for i in range(20000):
            self.adam.step(self.loss_func)
        self.optimizer.step(self.loss_func)

In [124]:
#PINN training
net = PINN()
net.train()

1000 0.0861930325627327
tensor([0.2701, 0.2911, 0.3128, 0.3352, 0.3584, 0.3823, 0.4071, 0.4326, 0.4590,
        0.4863, 0.5144, 0.5434, 0.5733, 0.6041, 0.6358, 0.6685, 0.7021, 0.7367,
        0.7722, 0.8087], device='cuda:0', grad_fn=<SliceBackward0>)
2000 0.026446877047419548
tensor([1.3743, 1.4532, 1.5345, 1.6183, 1.7044, 1.7927, 1.8832, 1.9757, 2.0699,
        2.1659, 2.2632, 2.3618, 2.4614, 2.5617, 2.6623, 2.7632, 2.8638, 2.9639,
        3.0631, 3.1610], device='cuda:0', grad_fn=<SliceBackward0>)
3000 0.015414269641041756
tensor([3.0057, 3.0972, 3.1867, 3.2741, 3.3592, 3.4417, 3.5217, 3.5988, 3.6729,
        3.7440, 3.8118, 3.8761, 3.9368, 3.9937, 4.0466, 4.0953, 4.1396, 4.1793,
        4.2142, 4.2442], device='cuda:0', grad_fn=<SliceBackward0>)
4000 0.006711466703563929
tensor([3.0899, 3.1769, 3.2619, 3.3447, 3.4252, 3.5033, 3.5790, 3.6522, 3.7227,
        3.7904, 3.8553, 3.9171, 3.9757, 4.0310, 4.0828, 4.1307, 4.1748, 4.2147,
        4.2502, 4.2811], device='cuda:0', grad_fn=<Sli

In [ ]:
#Saving the trained model
torch.save(net, './tm_lhs_8l_0.08m_0.1-0.0001.t7')

In [ ]:
#Post processing for results analysis

x = torch.arange(0, 3+0.01 ,0.01 )
y = torch.arange(0, 1+0.01 , 0.01)

x1 = torch.arange(1, 2+0.01 ,0.01 )
y1 = torch.arange(1, 2+0.01 , 0.01)

outlet = torch.stack(torch.meshgrid(x[-1], y)).reshape(2, -1).T
outlet1 = torch.stack(torch.meshgrid(x1, y1[-1])).reshape(2, -1).T
        
outlet = torch.cat([outlet, outlet1])

outlet = outlet.to(net.outlet.device)

outlet.requires_grad = True

TM = net.model
TM.eval()

y_pred = TM(outlet)
dpsi_dX = torch.autograd.grad(inputs=outlet, outputs=y_pred[:,0], grad_outputs=torch.ones_like(y_pred[:,0]), retain_graph=True, create_graph=True)[0]
dpsi_dX[:, 0] = -1*dpsi_dX[:, 0]  

fld=torch.cat([outlet, dpsi_dX ,y_pred], 1)
fld=fld.detach().cpu().numpy()

np.savetxt('output.txt', fld)